In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# ! pip install datasets transformers
!pip uninstall fsspec -qq -y
!pip install --no-index --find-links ../input/hf-datasets/wheels datasets -qq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cloud 0.1.13 requires tensorflow<3.0,>=1.15.0, which is not installed.
dask-cudf 21.6.1+2.g101fc0fda4 requires cupy-cuda112, which is not installed.
cudf 21.6.1+2.g101fc0fda4 requires cupy-cuda110, which is not installed.
s3fs 2021.6.1 requires fsspec==2021.06.1, but you have fsspec 2021.6.0 which is incompatible.
pytorch-lightning 1.3.8 requires fsspec[http]!=2021.06.0,>=2021.05.0, but you have fsspec 2021.6.0 which is incompatible.
dask-cudf 21.6.1+2.g101fc0fda4 requires dask<=2021.5.1,>=2021.4.0, but you have dask 2021.6.2 which is incompatible.
dask-cudf 21.6.1+2.g101fc0fda4 requires distributed<=2021.5.1,>=2.22.0, but you have distributed 2021.6.2 which is incompatible.


In [3]:
from datasets import load_dataset, Dataset
from pprint import pprint
import pandas as pd
import numpy as np

# Preprocessing

In [4]:
# model_checkpoint = "distilbert-base-uncased"
# model_checkpoint = "deepset/xlm-roberta-large-squad2"
# model_checkpoint = "../input/xlm-roberta-squad2/deepset/xlm-roberta-base-squad2"
model_checkpoint = "../input/pipeline-for-qa-train/test-chaii-trained"

from transformers import XLMTokenizer,AutoTokenizer
# tokenizer = XLMTokenizer.from_pretrained('xlm-mlm-en-2048')
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
tokenizer("What is your name?", "My name is Sylvain.")

{'input_ids': [0, 4865, 83, 935, 9351, 32, 2, 2, 2646, 9351, 83, 100973, 21845, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

# Doc_stride is used to handle large text: tokens>512

In [6]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

# Lodaing model

In [7]:
import gc
gc.collect()

119

In [8]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [9]:
trainer = Trainer(model)

# Real predictions

In [10]:
# val = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/test.csv')
# val_dataset = Dataset.from_pandas(val)

val = pd.read_csv('../input/chaii-hindi-and-tamil-question-answering/train.csv')[-64:]
val_dataset = Dataset.from_pandas(val)

In [11]:
val_dataset

Dataset({
    features: ['id', 'context', 'question', 'answer_text', 'answer_start', 'language'],
    num_rows: 64
})

In [12]:
# len(tokenized_example['input_ids'])

In [13]:
# In some padding required on left side
pad_on_right = tokenizer.padding_side == "right"
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [14]:
features = prepare_validation_features(val_dataset[:5])
print(features.keys())

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'example_id'])


In [15]:
validation_features = val_dataset.map(
    prepare_validation_features,
    batched=True,
    remove_columns=val_dataset.column_names
)

In [16]:
len(validation_features)

822

In [17]:
validation_features

Dataset({
    features: ['attention_mask', 'example_id', 'input_ids', 'offset_mapping'],
    num_rows: 822
})

In [18]:
raw_predictions = trainer.predict(validation_features)

In [19]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

# Top 1

In [20]:
from tqdm.auto import tqdm
import collections


def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
#         if not squad_v2:
        predictions[example["id"]] = best_answer["text"]
#         else:
#             answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
#             predictions[example["id"]] = answer

    return predictions

In [21]:
final_predictions = postprocess_qa_predictions(val_dataset, validation_features, raw_predictions.predictions)

Post-processing 64 example predictions split into 822 features.


In [22]:
final_list = list(final_predictions.values())
print(len(final_list))
# final_list

64


In [23]:
val.head()

,id,context,question,answer_text,answer_start,language
1050,60b6c1051,मणिपाल उच्च शिक्षा अकादमी भारत का एक समविश्ववि...,मणिपाल विश्वविद्यालय कहाँ स्तिथ है?,मणिपाल,0,hindi
1051,932c30d34,प्रोटीन या प्रोभूजिन एक जटिल भूयाति युक्त कार्...,सरल प्रोटीन का गठन किसके द्वारा होता है?,अमीनो अम्ल द्वारा,511,hindi
1052,78a4ff63f,गाम्बिया (आधिकारिक रूप से इस्लामी गणराज्य गाम्...,गाम्बिया' किस महाद्वीप में स्थित एक देश है?,अफ्रीकी,99,hindi
1053,01ae1bdd3,भारतीय विज्ञान की परंपरा विश्व की प्राचीनतम वै...,"हरगोविंद खुराना, किस पेशे से सम्बंधित थे?",भारतीय वैज्ञानिक,1932,hindi
1054,1acb37c63,क़ुस्तुंतुनिया या कांस्टैंटिनोपुल (यूनानी: Κων...,कांस्टैंटिनोपुल की स्थापना किस वर्ष में हुई थी?,328 ई.,596,hindi


In [24]:
val['PredictionString']=final_list

In [25]:
val.head()

,id,context,question,answer_text,answer_start,language,PredictionString
1050,60b6c1051,मणिपाल उच्च शिक्षा अकादमी भारत का एक समविश्ववि...,मणिपाल विश्वविद्यालय कहाँ स्तिथ है?,मणिपाल,0,hindi,कर्नाटक के मणिपाल
1051,932c30d34,प्रोटीन या प्रोभूजिन एक जटिल भूयाति युक्त कार्...,सरल प्रोटीन का गठन किसके द्वारा होता है?,अमीनो अम्ल द्वारा,511,hindi,अमीनो अम्ल
1052,78a4ff63f,गाम्बिया (आधिकारिक रूप से इस्लामी गणराज्य गाम्...,गाम्बिया' किस महाद्वीप में स्थित एक देश है?,अफ्रीकी,99,hindi,पश्चिमी अफ्रीका
1053,01ae1bdd3,भारतीय विज्ञान की परंपरा विश्व की प्राचीनतम वै...,"हरगोविंद खुराना, किस पेशे से सम्बंधित थे?",भारतीय वैज्ञानिक,1932,hindi,वैज्ञानिकों
1054,1acb37c63,क़ुस्तुंतुनिया या कांस्टैंटिनोपुल (यूनानी: Κων...,कांस्टैंटिनोपुल की स्थापना किस वर्ष में हुई थी?,328 ई.,596,hindi,328 ई.


In [26]:
def jaccard(row): 
    str1 = row['PredictionString']
    str2 = row['answer_text']
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [27]:
val['jaccard_score']=val.apply(jaccard,axis=1)

In [28]:
val.head(n=10)

,id,context,question,answer_text,answer_start,language,PredictionString,jaccard_score
1050,60b6c1051,मणिपाल उच्च शिक्षा अकादमी भारत का एक समविश्ववि...,मणिपाल विश्वविद्यालय कहाँ स्तिथ है?,मणिपाल,0,hindi,कर्नाटक के मणिपाल,0.333333
1051,932c30d34,प्रोटीन या प्रोभूजिन एक जटिल भूयाति युक्त कार्...,सरल प्रोटीन का गठन किसके द्वारा होता है?,अमीनो अम्ल द्वारा,511,hindi,अमीनो अम्ल,0.666667
1052,78a4ff63f,गाम्बिया (आधिकारिक रूप से इस्लामी गणराज्य गाम्...,गाम्बिया' किस महाद्वीप में स्थित एक देश है?,अफ्रीकी,99,hindi,पश्चिमी अफ्रीका,0.000000
1053,01ae1bdd3,भारतीय विज्ञान की परंपरा विश्व की प्राचीनतम वै...,"हरगोविंद खुराना, किस पेशे से सम्बंधित थे?",भारतीय वैज्ञानिक,1932,hindi,वैज्ञानिकों,0.000000
1054,1acb37c63,क़ुस्तुंतुनिया या कांस्टैंटिनोपुल (यूनानी: Κων...,कांस्टैंटिनोपुल की स्थापना किस वर्ष में हुई थी?,328 ई.,596,hindi,328 ई.,1.000000
1055,6ee60fcfb,"गांजा (Cannabis या marijuana), एक मादक पदार्थ ...",'गांजा'' व्यसन को अंग्रेज़ी में क्या कहा जाता है?,Cannabis या marijuana,7,hindi,marijuana,0.333333
1056,49accd67e,कुसुम (Carthamus tinctorius एल) एक अत्यधिक bra...,कुसुम पौधें का वैज्ञानिक नाम क्या है?,Carthamus tinctorius एल,7,hindi,Carthamus tinctorius एल,1.000000
1057,3f3c2c4c5,बिल्ली के कुल (विडाल) में आने वाला चीता (एसीनो...,चीता का वैज्ञानिक नाम क्या है?,एसीनोनिक्स जुबेटस,41,hindi,एसीनोनिक्स जुबेटस,1.000000
1058,ebe48c5e6,"इन्फोसिस लिमिटेड (BSE:, Nasdaq:) एक बहुराष्ट्र...",इंफोसिस लिमिटेड की स्थापना किस वर्ष में हुई थी?,"२ जुलाई, १९८१",475,hindi,"२ जुलाई, १९८१",1.000000
1059,9cf59a35c,एशियन पेंट्स लिमिटेड एक भारतीय बहुराष्ट्रीय कम...,एशियन पेंट्स कंपनी की स्थापना कब हुई?,फरवरी १९४२,521,hindi,\nफरवरी १९४२,1.000000


In [29]:
val['jaccard_score'].mean()

0.7109375

In [30]:
val.to_csv('jaccard_score.csv',index=False)

# Top 5

In [31]:
from tqdm.auto import tqdm
import collections


def postprocess_qa_predictions_topn(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30,top=1):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:top]
            best_answers = []
            for ans in best_answer:
                best_answers.append(ans['text'])
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answers = [""]
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
#         if not squad_v2:
#         print(best_answers)
        predictions[example["id"]] = best_answers
#         else:
#             answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
#             predictions[example["id"]] = answer

    return predictions

In [32]:
final_topn_predictions = postprocess_qa_predictions_topn(val_dataset, validation_features, raw_predictions.predictions,
                                                   top=5)

Post-processing 64 example predictions split into 822 features.


In [33]:
topn_final_list = list(final_topn_predictions.values())
print(len(topn_final_list))
val['topn_PredictionString']=topn_final_list

64


In [34]:
val.head()

,id,context,question,answer_text,answer_start,language,PredictionString,jaccard_score,topn_PredictionString
1050,60b6c1051,मणिपाल उच्च शिक्षा अकादमी भारत का एक समविश्ववि...,मणिपाल विश्वविद्यालय कहाँ स्तिथ है?,मणिपाल,0,hindi,कर्नाटक के मणिपाल,0.333333,"[ कर्नाटक के मणिपाल, कर्नाटक, कर्नाटक के मणि..."
1051,932c30d34,प्रोटीन या प्रोभूजिन एक जटिल भूयाति युक्त कार्...,सरल प्रोटीन का गठन किसके द्वारा होता है?,अमीनो अम्ल द्वारा,511,hindi,अमीनो अम्ल,0.666667,"[ अमीनो अम्ल, केवल अमीनो अम्ल, अमीनो अम्ल द्..."
1052,78a4ff63f,गाम्बिया (आधिकारिक रूप से इस्लामी गणराज्य गाम्...,गाम्बिया' किस महाद्वीप में स्थित एक देश है?,अफ्रीकी,99,hindi,पश्चिमी अफ्रीका,0.000000,"[ पश्चिमी अफ्रीका, अफ्रीका, पश्चिमी अफ्रीका ..."
1053,01ae1bdd3,भारतीय विज्ञान की परंपरा विश्व की प्राचीनतम वै...,"हरगोविंद खुराना, किस पेशे से सम्बंधित थे?",भारतीय वैज्ञानिक,1932,hindi,वैज्ञानिकों,0.000000,"[ वैज्ञानिकों, प्रमुख वैज्ञानिकों, विज्ञान, ..."
1054,1acb37c63,क़ुस्तुंतुनिया या कांस्टैंटिनोपुल (यूनानी: Κων...,कांस्टैंटिनोपुल की स्थापना किस वर्ष में हुई थी?,328 ई.,596,hindi,328 ई.,1.000000,"[ 328 ई., 328, 11 मई 330 ई., 328 ई. में, 3..."


In [35]:
def topn_jaccard(row): 
    strs = row['topn_PredictionString']
    str2 = row['answer_text']
    mx_score=0.0
    for str1 in strs:
        a = set(str1.lower().split()) 
        b = set(str2.lower().split())
        c = a.intersection(b)
        mx_score = max(float(len(c)) / (len(a) + len(b) - len(c)),mx_score)
    return mx_score

In [36]:
val['topn_jaccard_score']=val.apply(topn_jaccard,axis=1)
val.head(n=10)

,id,context,question,answer_text,answer_start,language,PredictionString,jaccard_score,topn_PredictionString,topn_jaccard_score
1050,60b6c1051,मणिपाल उच्च शिक्षा अकादमी भारत का एक समविश्ववि...,मणिपाल विश्वविद्यालय कहाँ स्तिथ है?,मणिपाल,0,hindi,कर्नाटक के मणिपाल,0.333333,"[ कर्नाटक के मणिपाल, कर्नाटक, कर्नाटक के मणि...",0.333333
1051,932c30d34,प्रोटीन या प्रोभूजिन एक जटिल भूयाति युक्त कार्...,सरल प्रोटीन का गठन किसके द्वारा होता है?,अमीनो अम्ल द्वारा,511,hindi,अमीनो अम्ल,0.666667,"[ अमीनो अम्ल, केवल अमीनो अम्ल, अमीनो अम्ल द्...",1.000000
1052,78a4ff63f,गाम्बिया (आधिकारिक रूप से इस्लामी गणराज्य गाम्...,गाम्बिया' किस महाद्वीप में स्थित एक देश है?,अफ्रीकी,99,hindi,पश्चिमी अफ्रीका,0.000000,"[ पश्चिमी अफ्रीका, अफ्रीका, पश्चिमी अफ्रीका ...",0.000000
1053,01ae1bdd3,भारतीय विज्ञान की परंपरा विश्व की प्राचीनतम वै...,"हरगोविंद खुराना, किस पेशे से सम्बंधित थे?",भारतीय वैज्ञानिक,1932,hindi,वैज्ञानिकों,0.000000,"[ वैज्ञानिकों, प्रमुख वैज्ञानिकों, विज्ञान, ...",0.500000
1054,1acb37c63,क़ुस्तुंतुनिया या कांस्टैंटिनोपुल (यूनानी: Κων...,कांस्टैंटिनोपुल की स्थापना किस वर्ष में हुई थी?,328 ई.,596,hindi,328 ई.,1.000000,"[ 328 ई., 328, 11 मई 330 ई., 328 ई. में, 3...",1.000000
1055,6ee60fcfb,"गांजा (Cannabis या marijuana), एक मादक पदार्थ ...",'गांजा'' व्यसन को अंग्रेज़ी में क्या कहा जाता है?,Cannabis या marijuana,7,hindi,marijuana,0.333333,"[ marijuana, Cannabis या marijuana, Cannabis, ...",1.000000
1056,49accd67e,कुसुम (Carthamus tinctorius एल) एक अत्यधिक bra...,कुसुम पौधें का वैज्ञानिक नाम क्या है?,Carthamus tinctorius एल,7,hindi,Carthamus tinctorius एल,1.000000,"[Carthamus tinctorius एल, एल, Carthamus tinct...",1.000000
1057,3f3c2c4c5,बिल्ली के कुल (विडाल) में आने वाला चीता (एसीनो...,चीता का वैज्ञानिक नाम क्या है?,एसीनोनिक्स जुबेटस,41,hindi,एसीनोनिक्स जुबेटस,1.000000,"[एसीनोनिक्स जुबेटस, जुबेटस, एसीनोनिक्स जुबेटस...",1.000000
1058,ebe48c5e6,"इन्फोसिस लिमिटेड (BSE:, Nasdaq:) एक बहुराष्ट्र...",इंफोसिस लिमिटेड की स्थापना किस वर्ष में हुई थी?,"२ जुलाई, १९८१",475,hindi,"२ जुलाई, १९८१",1.000000,"[ २ जुलाई, १९८१, \n1981, ८१, १९८१, जुलाई, १९८१]",1.000000
1059,9cf59a35c,एशियन पेंट्स लिमिटेड एक भारतीय बहुराष्ट्रीय कम...,एशियन पेंट्स कंपनी की स्थापना कब हुई?,फरवरी १९४२,521,hindi,\nफरवरी १९४२,1.000000,"[\nफरवरी १९४२, ४२, १९४२, \nफरवरी, इतिहास \nफ...",1.000000


In [37]:
val['topn_jaccard_score'].mean()

0.8895833333333334

In [38]:
val.to_csv('topn_jaccard_score.csv',index=False)